In [1]:
# data cleaning and formatting
import numpy as np 
import pandas as pd
from sklearn.neighbors import KernelDensity

# map visualisation
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from PIL import Image, ImageDraw

#widgets
import ipywidgets as widgets
from ipywidgets import interact_manual, interact, Layout, Box, Label, Checkbox
from IPython import display as d
import re

from IPython.core.display import HTML

pd.options.mode.chained_assignment = None  # default='warn'

--- 
 
 
# Data Processing and Cleaning 
 
> We start by adjusting the data from the dataframe to better fit our requirements. Note that the dataframe has already been cleaned of outliers before, hence we do not have to do it again. 


In [2]:
def adjust_data(df):
    
    df = df[(df['victim_position_x'] != 0) | (df['victim_position_y'] != 0)]
    df = df[(df['killer_position_x'] != 0) | (df['killer_position_y'] != 0)]
    return df

def adjust_coor(df):
    coordinate_columns = ['victim_position_x', 'victim_position_y',
                          'killer_position_x', 'killer_position_y']
    df[coordinate_columns] = df[coordinate_columns] / 800000 * 4096
    return df

def adjust_time(df):
    time_column = ['time']
    df[time_column] = round(df[time_column] / 60)
    return df

df = pd.read_csv('sample.csv')
df = adjust_data(df)
df = adjust_coor(df)
df = adjust_time(df)

In [3]:
%%html
<style>
table {float:left}
</style>

---
 
 
# Functions
 
Here we create <b>functions</b> and <b>variables</b> crucial for our visualisation

| Name of Function | What does it do | 
| :-- | :-- | 
| image | Holds the image of the map erangel | 
| heatmap | Generates and outputs the map | 
| final list | List of every cause of death in PubG  |
| euclidian distance |  Find distance between two points  | 

In [4]:
#defining image
image = Image.open('erangel.jpg')
sniper = Image.open('sniper.gif')

#contour colour and gradient
contour_colour = np.zeros((50, 4))
contour_colour[:, 0] = np.linspace(0, 1, 50)
contour_colour[:, 3] = np.linspace(0.5, 1, 50)
contour_colour[:10] = 0
contour_colour_cmap = mcolors.LinearSegmentedColormap.from_list('alpha_reds', 
                                                                colors = contour_colour, 
                                                                N = 4096, 
                                                                gamma = 1)

#generating map
def heatmap(df,
            img,
            bw = 80,
            cmap = plt.cm.Reds,
            ax = None):
    
    x_mesh = np.linspace(0, 4096, 100)
    y_mesh = np.linspace(0, 4096, 100)
    X, Y = np.meshgrid(x_mesh, y_mesh)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    
    if(len(df)==0):
        display(img)
        return 0
    
    kde = KernelDensity(kernel = 'epanechnikov', bandwidth = bw).fit(df.to_numpy())
    log_density = kde.score_samples(xy)
    log_density = log_density.reshape((100, 100))
    density = np.exp(log_density)

    if ax is None:
        fig, ax = plt.subplots(figsize=(16, 16), clear=True)

    plt.gca().invert_yaxis()
    ax.contourf(X, Y, density, alpha = None, cmap = cmap)
    ax.imshow(img)
    return fig, ax

#list of cause of death
weapons = df['killed_by'].unique().tolist()
final_list = []

for i in weapons:
    x = re.search("^death|BP", i)
    if(x):
        weapons.remove(i)
for i in weapons:
    if (i.find("death") == -1):
        final_list.append(i)

final_list.sort()
final_list.remove('Crossbow')
final_list.insert(0,'Shotgun/Pistol')
final_list.insert(0,'SMG')
final_list.insert(0,'Rifle')
final_list.insert(0,'Sniper')
final_list.insert(0,'All')

def euclidian_distance(a, b): 
    distance = np.sqrt(((a - b)**2).sum(axis=1))
    return distance


--- 
 
 
# Visualisation
 
> #### Forming the User Interface by using widgets from ipywidgets library. The widgets act as a filter for the user to see specific data.
   
>We use <b>slider</b> for time, <b>dropbox</b> for cause of death and the <b>checkbox</b> to filter distance specific kills; formatting them with the 'Box' function
> 
>> We also create the function <b>newmap</b> to update the map when user makes a new input to the filters provided.

Users are to select the specific time, cause of death and distance. Then, click on the 'Run Interact' button to see the spots where people are highly likely to die. The redder the spot, higher the number of death in that area.

In [5]:

#slider for mins
w = widgets.IntSlider(
    value=1,
    min=1,
    max=36,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

#dropdown box for cause of death
dd = widgets.Dropdown(
    options = final_list,
    value ='All',
)

cb = widgets.Checkbox(
    False, 
    description='',
    indent = False)


#Formatting both widgets together
form_item_layout = Layout(
    display='flex',
    flex_flow='row'
)

form_items = [Box([Label(value='Which Min: '), w], layout=form_item_layout),
              Box([Label(value='Killed By:'), dd], layout=form_item_layout),
              Box([Label(value='Filter by Long Range Kills:'), cb], layout=form_item_layout)]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 1px',
    align_items='stretch',
    width='50%'
))

display(form) #display widgets

def new_map():
        data_df = df.loc[df['time'] == w.value]
        
        if (cb.value == False):
            
            if(dd.value == 'Sniper'):
                sniper = data_df[data_df["killed_by"].str.contains("AWM|Kar98k|M24|Mini 14|Mk14|SKS|SLR")==True]
                map_df = sniper[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)
            
            elif(dd.value == 'Rifle'): 
                rifle = data_df[data_df["killed_by"].str.contains("AUG A3|AKM|DP-28|Groza|G36C|M16A4|M249|M416|M762|MK47 Mutant|QBU|QBZ95|Scar|VSS|Win94")==True]
                map_df = rifle[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)
            
            elif(dd.value == 'SMG'):
                smg = data_df[data_df["killed_by"].str.contains("Bizon|MP5K|Micro UZI|Tommy Gun|UMP45|Vector")==True]
                map_df = smg[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)
            
            elif(dd.value == 'Shotgun/Pistol'):
                shotgun_pistol = data_df[data_df["killed_by"].str.contains("P18C|P1911|P92|R1895|R45|S12K|S1897|S686|Sawed-off|Skorpion")==True] 
                map_df = shotgun_pistol[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)
            
            elif(dd.value != 'All'):
                data_df = data_df.loc[data_df['killed_by'] == dd.value]
                map_df = data_df[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)
            
            else:    
                map_df = data_df[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)
  #-------------------------------------------------------------------------------------------------     
        else:
            dup_df = data_df
            dup_df['distance'] = euclidian_distance(dup_df[['killer_position_x', 'killer_position_y']].to_numpy(),
                                                 dup_df[['victim_position_x', 'victim_position_y']].to_numpy())
            longrange_df = dup_df[dup_df['distance'] > dup_df['distance'].quantile(0.8)]
                                    
            if(dd.value == 'Sniper'):
                sniper = longrange_df[longrange_df["killed_by"].str.contains("AWM|Kar98k|M24|Mini 14|Mk14|SKS|SLR")==True]
                map_df = sniper[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)
            
            elif(dd.value == 'Rifle'): 
                rifle = longrange_df[longrange_df["killed_by"].str.contains("AUG A3|AKM|DP-28|Groza|G36C|M16A4|M249|M416|M762|MK47 Mutant|QBU|QBZ95|Scar|VSS|Win94")==True]
                map_df = rifle[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)
            
            elif(dd.value == 'SMG'):
                smg = longrange_df[longrange_df["killed_by"].str.contains("Bizon|MP5K|Micro UZI|Tommy Gun|UMP45|Vector")==True]
                map_df = smg[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)
            
            elif(dd.value == 'Shotgun/Pistol'):
                shotgun_pistol = longrange_df[longrange_df["killed_by"].str.contains("P18C|P1911|P92|R1895|R45|S12K|S1897|S686|Sawed-off|Skorpion")==True] 
                map_df = shotgun_pistol[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)
            
            elif(dd.value != 'All'):
                longrange_df = longrange_df.loc[longrange_df['killed_by'] == dd.value]
                map_df = data_df[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)
            
            else:    
                map_df = longrange_df[['victim_position_x', 'victim_position_y']]
                heatmap(map_df, image, cmap=contour_colour_cmap)

interact_manual(new_map);

Box(children=(Box(children=(Label(value='Which Min: '), IntSlider(value=1, continuous_update=False, max=36, mi…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

--- 
 
 
# Conclusion
 
> #### Through the visualisation, we identified some points of interest.
<h2>General Pointers</h2>

<font color = maroon>1. There is a general spike in deaths by all weapons from the 12th min onwards.</font>

<font color = blue>2. Sniper rifles and Rifles are the popular choices to use as compared to the SMG and Shotgun/Pistol.</font>

<font color = red>3. The area around the town of Georgopol is the most likely spot to get killed.</font>

<h2>Weapon Specific Points</h2>

<font color = green>1. SMG seems to be a popular choice early in the game, but most players opt for other types of weapons from the 10th min onwards.</font>